# Introduction to running PROCESS

A Jupyter notebook to demonstrate usage of the `process` package.

## Motivation
Process is moving away from being a runnable package with a single command-line entry-point to an importable package which can be scripted. This notebook is a good way of demonstrating the functionality of the package, and could provide a better way of working for modellers, who may wish to create their own scripts or notebooks for different tasks.

## Setup
Currently the various classes and "utilities" scripts in Process have different interfaces and read and write files in differing manners and in locations that can't be easily controlled. To partially avoid the headaches associated with this, the code cell below defines a function to allow each example to be run in a temporary directory, much like a test. Input files are copied to this temporary directory and outputs contained there before the directory is removed.

This temporary directory function is only required for running the examples below and removing any modifications afterwards, not in regular use of Process where the outputs will want to be preserved. Further development work will unify these disparate ways of running Process into a common Pythonic form.

In [1]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
from shutil import copy
from tempfile import TemporaryDirectory

# Define project root dir; when running a notebook, the cwd is the dir the notebook is in
PROJ_DIR = Path.cwd().parent


def copy_to_temp_dir(input_rel):
    """Copy an input file to a new temp dir and return its new path.

    The new TemporaryDirectory object is returned to avoid destruction of the
    object, which results in deletion of the directory prematurely. This way
    the cleanup() method can be used to delete the directory when required.
    :param input_rel: file path relative to project root dir
    :type input_rel: str
    :return: temporary dir and absolute path to file in temp dir
    :rtype: (TemporaryDirectory, pathlib.Path)
    """
    # Create temporary dir to contain the run's outputs
    temp_dir = TemporaryDirectory()
    temp_dir_path = Path(temp_dir.name)

    # Define absolute path for input file
    input_rel_path = Path(input_rel)
    input_abs_path = PROJ_DIR / input_rel_path

    try:
        assert input_abs_path.exists()
    except AssertionError as err:
        raise FileNotFoundError("Input file doesn't exist.") from err

    # Copy input file to temp dir
    copy(input_abs_path, temp_dir_path)
    temp_input_path = temp_dir_path / input_abs_path.name

    return temp_dir, temp_input_path


## Basic run of Process
Run Process on an input file using the `SingleRun` class. This outputs an `MFILE.DAT` and an `OUT.DAT`.

In [2]:
from process.main import SingleRun, setup_loggers

# setup the loggers so that the output is not spammed with model errors/warnings
setup_loggers()

# Define input file name relative to project dir, then copy to temp dir
script_dir = Path("__file__").parent.resolve()
input_rel = script_dir / "data/large_tokamak_IN.DAT"

temp_dir, temp_input_path = copy_to_temp_dir(input_rel)

# Run process on an input file in a temporary directory
single_run = SingleRun(str(temp_input_path))
single_run.run()

The IN.DAT file does not contain any obsolete variables.
 
**************************************************************************************************************
************************************************** PROCESS ***************************************************
************************************** Power Reactor Optimisation Code ***************************************
**************************************************************************************************************
 
Version : 3.2.1
Git Tag : 
Git Branch : 
Date : 13/11/2025 UTC
Time : 14:10
User : jaebeomcho
Computer : Jaebeoms-MacBook-Pro-4.local
Directory : /Users/jaebeomcho/Desktop/code/PROCESS/examples
Input : /private/var/folders/hj/kc7f02zx6sddr7fxvgyx78xw0000gn/T/tmpi_k7ul6r/large_tokamak_IN.DAT
Run title : Generic large tokamak
Run type : Reactor concept design: Pulsed tokamak model, (c) UK Atomic Energy Authority
 
*******************************************************************************

/Users/jaebeomcho/Desktop/code/PROCESS/process/init.py:92: UserWarning: Lower limit of volume averaged electron temperature (temp_plasma_electron_vol_avg_kev) has been raised to ensure temp_plasma_electron_vol_avg_kev > temp_plasma_pedestal_kev
  check_process(inputs)
/Users/jaebeomcho/Desktop/code/PROCESS/process/init.py:92: UserWarning: temp_cs_superconductor_margin_min and tmargmin should not both be specified in IN.DAT temp_cs_superconductor_margin_min has been ignored
  check_process(inputs)
/Users/jaebeomcho/Desktop/code/PROCESS/process/physics.py:7711: RuntimeWarning: divide by zero encountered in scalar divide
  * (nd_plasma_pedestal_electron / n_greenwald) ** -0.174
/Users/jaebeomcho/Desktop/code/PROCESS/process/costs.py:3030: RuntimeWarning: invalid value encountered in sqrt
  annoam = cost_variables.ucoam[cost_variables.lsa - 1] * np.sqrt(
/Users/jaebeomcho/Desktop/code/PROCESS/process/costs.py:3101: RuntimeWarning: invalid value encountered in sqrt
  annwst = cost_variables

8 | Convergence Parameter: 4.647E-11
 
************************************* PROCESS found a feasible solution **************************************
 
******************************************** Errors and Warnings *********************************************
(/Users/jaebeomcho/Desktop/code/PROCESS/process/physics.py:6653) Diamagnetic fraction is more than 1%, but not calculated. Consider using i_diamagnetic_current=2 and i_pfirsch_schluter_current=1

(/Users/jaebeomcho/Desktop/code/PROCESS/process/tf_coil.py:246) dr_tf_plasma_case to small to accommodate the WP, forced to minimum value

(/Users/jaebeomcho/Desktop/code/PROCESS/process/pfcoil.py:2385) CS not using max current density: further optimisation may be possible
 
******************************************* End of PROCESS Output ********************************************
 


## Plot summary
Create a summary PDF of the generated `MFILE.DAT` using `plot_proc`.

In [3]:
import subprocess

from pdf2image import convert_from_path

from process.io import plot_proc

# plot_proc uses command line arguments of the current process. Jupyter adds command line arguments under the hood causing plot_proc to fail. running plot proc in its own process isolates it from the jupyter command line arguments
subprocess.run(["python", plot_proc.__file__, "-f", str(single_run.mfile_path)])

# Create a summary PDF
# Convert PDF to PNG in order to display in notebook
summary_pdf = str(single_run.mfile_path) + "SUMMARY.pdf"
print(summary_pdf)
pages = convert_from_path(summary_pdf)
for page_no, page_image in enumerate(pages):
    png_path = script_dir / f"plot_proc_{page_no + 1}.png"
    page_image.save(png_path, "PNG")

/Users/jaebeomcho/Desktop/code/PROCESS/process/io/plot_proc.py:12375: DeprecationWarning: path is deprecated. Use files() instead. Refer to https://importlib-resources.readthedocs.io/en/latest/using.html#migrating-from-legacy for migration advice.
  with resources.path(
/Users/jaebeomcho/Desktop/code/PROCESS/process/io/plot_proc.py:2785: DeprecationWarning: path is deprecated. Use files() instead. Refer to https://importlib-resources.readthedocs.io/en/latest/using.html#migrating-from-legacy for migration advice.
  with resources.path(
/Users/jaebeomcho/Desktop/code/PROCESS/process/io/plot_proc.py:2829: DeprecationWarning: path is deprecated. Use files() instead. Refer to https://importlib-resources.readthedocs.io/en/latest/using.html#migrating-from-legacy for migration advice.
  with resources.path("process.io", "neutron.png") as neutron_image_path:
/Users/jaebeomcho/Desktop/code/PROCESS/process/io/plot_proc.py:402: DeprecationWarning: path is deprecated. Use files() instead. Refer to 

/var/folders/hj/kc7f02zx6sddr7fxvgyx78xw0000gn/T/tmpi_k7ul6r/large_tokamak_MFILE.DATSUMMARY.pdf


PDFInfoNotInstalledError: Unable to get page count. Is poppler installed and in PATH?

`plot_proc`'s PDF output.

In [ ]:
from IPython.display import display  # noqa: A004
from PIL import Image

img1 = Image.open("plot_proc_1.png")
display(img1)
img2 = Image.open("plot_proc_2.png")
display(img2)
img3 = Image.open("plot_proc_3.png")
display(img3)

In [ ]:
# Delete temp dir
temp_dir.cleanup()

## View key output variables
Run the large tokamak scenario using `SingleRun` to set some values on the `CostModel` instance and then print them.

In [ ]:
# Define input file name relative to project dir
input_rel = script_dir / "data/large_tokamak_IN.DAT"
print(input_rel)
temp_dir, temp_input_path = copy_to_temp_dir(input_rel)

# Run process on an input file
single_run = SingleRun(str(temp_input_path))
single_run.run()

In [ ]:
import process.data_structure

# Print some values on the CostModel instance
print(f"Heat transport system: {process.data_structure.cost_variables.c226:.3e} M$")
print(f"Electrical plant equipment: {process.data_structure.cost_variables.c24:.3e} M$")

In [ ]:
# Clean up
temp_dir.cleanup()

## VaryRun
Vary iteration parameters until a feasible solution is found, using the `VaryRun` class.

In [ ]:
import os

from process.main import VaryRun

input_rel = script_dir / "data/run_process.conf"
temp_dir, temp_input_path = copy_to_temp_dir(input_rel)

# .conf file relies on a separate input file too; copy this as well
# TODO This double input file requirement needs to be removed
input_rel_2 = script_dir / "data/large_tokamak_IN.DAT"
copy(PROJ_DIR / input_rel_2, temp_dir.name)

# VaryRun uses process_config.py, which changes the current working directory
# via os.chdir() to the temporary dir. Apart from being bad practice, once the
# temp dir is removed, this causes Path.cwd() (as used in plot_scans.py) to
# throw an exception when trying to return the (now deleted) CWD. Hence it
# needs to be set back after VaryRun()
# TODO Remove the os.chdir() from VaryRun
cwd = Path.cwd()

vary_run = VaryRun(str(temp_input_path))
vary_run.run()
os.chdir(cwd)

temp_dir.cleanup()